In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult'
if not op.exists(mDIR):
    mkdir(mDIR)

# spatial autocorrelation by treatment - adult x adult - by tx

### put into R
```R
source('~/imports.R')
library(PopGenReport)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.tx <- getdf(treatment='tx')
head(popgen.tx)
pg.tx <- popgen.tx[,chloro]
head(pg.tx)

### get a dataframe for each plot
adults <- simpmoran[which(simpmoran$seedling == "False"),'samp']
plots <- uni(popgen.tx$pop)[1:3]
dfs <- c()
for (plot in plots){
    df <- popgen.tx[which(popgen.tx$pop == plot),]
    df <- df[which(rownames(df) %in% adults),]
    assign(sprintf("%spop",plot),df$pop)
    df <- df[,chloro]
    p <- plot
    assign(p,df)
    head(df)
}

### get genind objects
geninds <- c()
for (plot in plots){
    df      <- get(plot)
    pops    <- get(sprintf("%spop",plot))
    genind  <- sprintf("gen%s",plot)
    geninds <- c(geninds,genind)
    assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}
genuc
genbc
genun

### get gd objects
gdnames = c()
for (genind in geninds){
    gdname <- sprintf("gd%s",genind)
    gdnames <- c(gdnames,gdname)
    gd <- gd.smouse(get(genind))
    gd <- as.matrix(gd)
    assign(gdname,gd)
}
# check dimensions
for (gdn in gdnames){
    gd <- get(gdn)
    print(gdn)
    print(dim(gd))
}

### get euclidian distance
len(which(is.na(distmat))) == 0

### from full distmat, create smaller distmats based on treatment type and size class
simpmoran[,'plot']      <- substr(simpmoran$samp,1,3)
simpmoran[,'treatment'] <- substr(simpmoran$plot,1,2)
simpmoran[,'plot_rep']  <- substr(simpmoran$plot,3,3)

txs <- uni(simpmoran$treatment)[1:3]
distnames <- c()
for (tx in txs){
    print(tx)
    gdname <- sprintf("gdgen%s",tx)
    gd <- get(gdname)
    sampnames <- rownames(gd) # adults in treatment
    d <- distmat[which(rownames(distmat) %in% sampnames),which(colnames(distmat) %in% sampnames)]
    print(dim(d))
    distname  <- sprintf('dist%s',tx)
    distnames <- c(distnames,distname)
    assign(distname,d)
}

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
    print(gd)
    gdist <- get(gd)
    tx <- substr(gd,6,7)
    d <- get(sprintf("dist%s",tx))
    bins = ceiling(.max(d)/10)
    binname = sprintf("bin%s",gd)
    assign(binname,bins)
    autobins <- c(autobins,binname)
    auto <- spautocor(gdist,d,bins=bins)
    autoname <- sprintf("auto%s",gd)
    autos <- c(autos,autoname)
    assign(autoname,auto)
}

### print out range in r
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (range(a$r))
}
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (.range(a$r))
}


### save environment
DIR = '~/fragment/postmoran/by_tx/size_class/chloro/'
if (!(file.exists(DIR))){
    dir.create(DIR)
    print('created DIR')
}
DIR = '~/fragment/postmoran/by_tx/size_class/chloro/adultxadult'
if (!(file.exists(DIR))){
    dir.create(DIR)
    print('created DIR')
}
cd(DIR)
rds()
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp /Users/Brandon1/fragment/postmoran/by_tx/size_class/chloro/adultxadult/*RDS godel:/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/final

### multiprocess null confidence intervals

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult'
dirs = ['bc',
        'uc',
        'un',
        'us']
for d in dirs:
    d = op.join(DIR,d)
    if not op.exists(d):
        mkdir(d)

In [ ]:
dirs = []
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult'
ds = ['bc',
        'uc',
        'un',
        'us']
for d in ds:
    dirs.append(op.join(DIR,d))
    assert op.exists(op.join(DIR,d))
dirs

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# get plot IDs
txs = []
for d in dirs:
    if not 'final' in d:
        txs.append(op.basename(d))
txs = txs[:-1]
txs

In [ ]:
# write scripts per tx to get 999 null values of spatial autocorrelation
for tx in txs:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % tx,
           'dist%s' % tx,
           'bingdgen%s' % tx,
           tx,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/rfiles/null_%s.R' % (tx,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# get .R files
rfiles = [] # redundant so I don't use wrong rfiles
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
assert len(rfiles) == 2997
ceil = math.ceil(len(rfiles)/195)

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
cd ('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/shfiles/')
!ls *sh | parallel qsub {}

In [ ]:
# check on progress
for tx in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/outfiles/' % tx
    assert op.exists(DIR)
    print tx,len(ls(DIR))

In [ ]:
# combine null runs into a couple objects
for tx in txs:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/final')                      ###############
getrds()
tx = '%s'

sauto <- get(sprintf('autogdgen%%s',tx))
bins  <- get(sprintf('bingdgen%%s',tx))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%%s/outfiles',tx) ###############
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',tx))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%%s',tx))       ####################
filE = fname(getwd(),sprintf('%%s_chloro_null.RDS',tx))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_chloro_R.RDS',tx))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_chloro_Rm.RDS',tx))
saveRDS(Rm,filE)

''' % (tx)
    filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    assert op.exists(f)
    rfiles.append(f)
rfiles

In [ ]:
shfiles = []
for f in rfiles:
    tx = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (tx,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult/combine'
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % tx)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
cd(op.dirname(shfiles[0]))
cwd()
!ls

In [ ]:
for s in shfiles:
    cd(op.dirname(s))
    !qsub $s

In [ ]:
# scp these files to computer
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxadult'
prefs = ['autogdgen','bingdgen']
files = []
for tx in txs:
    d = op.join(DIR,tx)
    assert op.exists(d)
    for p in prefs:
        filE = op.join(op.join(DIR,'final'),'%s%s.RDS' % (p,tx))
        assert op.exists(filE)
        files.append(filE)
    filE = op.join(d,'%s_chloro_null.RDS' % tx)
    assert op.exists(filE)
    files.append(filE)
files

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/chloro_by_tx_adultxadult')
getrds()
txs <- c('bc','uc','un')

Lows <- list()
His  <- list()
for (tx in txs){
    null <- get(sprintf('%s_chloro_null',tx))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- tx
    names(His)[len(His)]   <- tx
}
```
### then use code below to plot figs

In [ ]:
# print out text to paste into R
for tx in txs:
    text = '''
########
tx = '%s'
his  <- His[[sprintf('%%s',tx)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',tx)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',tx))
sauto[which(is.na(sauto$r)),'r'] <- 0
cutoff <- which(sauto$bin > 325)
dev.new(width=3.25,height=3.25)
par(cex.lab=1,cex.axis=1)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="",
    ylab="",
    ylim=c(-0.2,0.2),
    xaxt="n",
    main="")
xtick <- seq(0,max(sauto$bin),200)
axis(side=1,at=xtick)
#polygon(c(rev(sauto$bin[-cutoff]),sauto$bin[-cutoff]),c(rev(his[-cutoff]),lows[-cutoff]),col='grey75',border=FALSE)
#lines(sauto$bin,sauto$r,lwd=2)
#lines(sauto$bin[-cutoff],lows[-cutoff],col="red",lty=2)
#lines(sauto$bin[-cutoff],his[-cutoff],col="red",lty=2)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
####
dev.new(width=10,height=10)
par(cex.lab=1.5,cex.axis=1.5)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.2,0.2),
    xlim=c(0,300),
    xaxt="n",
    xaxt='n') #    main=sprintf('%%s',tx),
xtick <- seq(0,max(sauto$bin),25)
axis(side=1,at=xtick,FALSE)
axis(side=1,at=xtick[xtick %%%% 50 ==0],labels=xtick[xtick %%%% 50 ==0])
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col=grey.colors(15)[5],border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
abline(v=xtick,col='grey',lwd=0.5)
########
''' % tx
    print text

In [ ]:
# scp ~/fragment/postmoran/by_tx_chloro_adultxadult_* godel:/home/lindb/teakettle/postmoran/figs

In [ ]:
from wand.image import Image as WImage
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_bc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_bc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_uc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_uc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_un.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxadult_un_inset.pdf')
img

# spatial autocorrelation chloro by tx - seedling x seedling - by tx

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed'
if not op.exists(mDIR):
    print 'created'
    mkdir(mDIR)

### put into R
```R
source('~/imports.R')
library(PopGenReport)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.tx <- getdf(treatment='tx')
head(popgen.tx)
pg.tx <- popgen.tx[,chloro]
head(pg.tx)

### get a dataframe for each plot
seedlings <- simpmoran[which(simpmoran$seedling == "True"),'samp']
plots <- uni(popgen.tx$pop)[1:3]
dfs <- c()
for (plot in plots){
    df <- popgen.tx[which(popgen.tx$pop == plot),]
    df <- df[which(rownames(df) %in% seedlings),]
    assign(sprintf("%spop",plot),df$pop)
    df <- df[,chloro]
    p <- plot
    assign(p,df)
}

### get genind objects
geninds <- c()
for (plot in plots){
    df      <- get(plot)
    pops    <- get(sprintf("%spop",plot))
    genind  <- sprintf("gen%s",plot)
    geninds <- c(geninds,genind)
    assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}
genuc
genbc
genun

### get gd objects
gdnames = c()
for (genind in geninds){
    gdname <- sprintf("gd%s",genind)
    gdnames <- c(gdnames,gdname)
    gd <- gd.smouse(get(genind))
    gd <- as.matrix(gd)
    assign(gdname,gd)
}
# check dimensions
for (gdn in gdnames){
    gd <- get(gdn)
    print(gdn)
    print(dim(gd))
}

### get euclidian distance
len(which(is.na(distmat)))

### from full distmat, create smaller distmats based on treatment type and size class
simpmoran[,'plot']      <- substr(simpmoran$samp,1,3)
simpmoran[,'treatment'] <- substr(simpmoran$plot,1,2)
simpmoran[,'plot_rep']  <- substr(simpmoran$plot,3,3)

txs <- uni(simpmoran$treatment)[1:3]
distnames <- c()
for (tx in txs){
    print(tx)
    gdname <- sprintf("gdgen%s",tx)
    gd <- get(gdname)
    sampnames <- rownames(gd) # seedlings in treatment
    d <- distmat[which(rownames(distmat) %in% sampnames),which(colnames(distmat) %in% sampnames)]
    print(dim(d))
    distname  <- sprintf('dist%s',tx)
    distnames <- c(distnames,distname)
    assign(distname,d)
}

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
    print(gd)
    gdist <- get(gd)
    tx <- substr(gd,6,7)
    d <- get(sprintf("dist%s",tx))
    bins = ceiling(.max(d)/10)
    binname = sprintf("bin%s",gd)
    assign(binname,bins)
    autobins <- c(autobins,binname)
    auto <- spautocor(gdist,d,bins=bins)
    autoname <- sprintf("auto%s",gd)
    autos <- c(autos,autoname)
    assign(autoname,auto)
}

### print out range in r
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (range(a$r))
}
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (.range(a$r))
}
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (len(a$r))
}


### save environment
DIR = '~/fragment/postmoran/by_tx/size_class/chloro/seedxseed'
if (!(file.exists(DIR))){
    dir.create(DIR)
    print('created DIR')
}
cd(DIR)
rds()

```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp ~/fragment/postmoran/by_tx/size_class/chloro/seedxseed/*RDS godel:/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/final


In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed'
dirs = ['bc',
        'uc',
        'un',
        'us']
for d in dirs:
    d = op.join(DIR,d)
    if not op.exists(d):
        mkdir(d)
dirs = []
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed'
ds = ['bc',
        'uc',
        'un',
        'us']
for d in ds:
    dirs.append(op.join(DIR,d))
    assert op.exists(op.join(DIR,d))
dirs

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# write scripts per tx to get 999 null values of spatial autocorrelation
for tx in txs:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % tx,
           'dist%s' % tx,
           'bingdgen%s' % tx,
           tx,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/rfiles/null_%s.R' % (tx,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil
# get .R files
rfiles = [] # redundant so I don't use wrong rfiles
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
assert len(rfiles) == 2997

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
cd ('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/shfiles/')
!ls *sh | parallel qsub {}

In [ ]:
# check on progress
for tx in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/outfiles/' % tx
    assert op.exists(DIR)
    print tx,len(ls(DIR))

In [ ]:
# combine null runs into a couple objects
for tx in txs:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/final')                      ###############
getrds()
tx = '%s'

sauto <- get(sprintf('autogdgen%%s',tx))
bins  <- get(sprintf('bingdgen%%s',tx))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%%s/outfiles',tx) ###############
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',tx))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%%s',tx))       ####################
filE = fname(getwd(),sprintf('%%s_chloro_null.RDS',tx))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_chloro_R.RDS',tx))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_chloro_Rm.RDS',tx))
saveRDS(Rm,filE)

''' % (tx)
    filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    rfiles.append(f)
rfiles

In [ ]:
shfiles = []
for f in rfiles:
    tx = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (tx,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed/combine'
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % tx)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
cd(op.dirname(shfiles[0]))
cwd()

In [ ]:
for s in shfiles:
    cd(op.dirname(s))
    !qsub $s

In [ ]:
# scp these files to computer
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/seedxseed'
prefs = ['autogdgen','bingdgen']
files = []
for tx in txs:
    d = op.join(DIR,tx)
    assert op.exists(d)
    for p in prefs:
        filE = op.join(op.join(DIR,'final'),'%s%s.RDS' % (p,tx))
        assert op.exists(filE)
        files.append(filE)
    filE = op.join(d,'%s_chloro_null.RDS' % tx)
    assert op.exists(filE)
    files.append(filE)
files

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/chloro_by_tx_seedxseed')
getrds()
txs <- c('bc','uc','un')

Lows <- list()
His  <- list()
for (tx in txs){
    null <- get(sprintf('%s_chloro_null',tx))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- tx
    names(His)[len(His)]   <- tx
}
```
### then use code below to plot figs

In [ ]:
##### print out text to paste into R
for tx in txs:
    text = '''
########
tx = '%s'
his  <- His[[sprintf('%%s',tx)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',tx)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',tx))
sauto[which(is.na(sauto$r)),'r'] <- 0
cutoff <- which(sauto$bin > 325)
dev.new(width=3.25,height=3.25)
par(cex.lab=1,cex.axis=1)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="",
    ylab="",
    ylim=c(-0.2,0.2),
    xaxt="n",
    main="")
xtick <- seq(0,max(sauto$bin),200)
axis(side=1,at=xtick)
#polygon(c(rev(sauto$bin[-cutoff]),sauto$bin[-cutoff]),c(rev(his[-cutoff]),lows[-cutoff]),col='grey75',border=FALSE)
#lines(sauto$bin,sauto$r,lwd=2)
#lines(sauto$bin[-cutoff],lows[-cutoff],col="red",lty=2)
#lines(sauto$bin[-cutoff],his[-cutoff],col="red",lty=2)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
####
dev.new(width=10,height=10)
par(cex.lab=1.5,cex.axis=1.5)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.2,0.2),
    xlim=c(0,300),
    xaxt="n",
    xaxt='n') #    main=sprintf('%%s',tx),
xtick <- seq(0,max(sauto$bin),25)
axis(side=1,at=xtick,FALSE)
axis(side=1,at=xtick[xtick %%%% 50 ==0],labels=xtick[xtick %%%% 50 ==0])
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col=grey.colors(15)[5],border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
abline(v=xtick,col='grey',lwd=0.5)
########
''' % tx
    print text

In [ ]:
# scp ~/fragment/postmoran/by_tx_chloro_seedxseed_* godel:/home/lindb/teakettle/postmoran/figs

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_bc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_bc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_uc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_uc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_un.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_seedxseed_un_inset.pdf')
img

# spatial autocorrelation chloro - adult x seed - by tx

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed'
if not op.exists(mDIR):
    mkdir(mDIR)

### put in to R
```R
source('~/imports.R')
library(PopGenReport)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.tx <- getdf(treatment='tx')
head(popgen.tx)
pg.tx <- popgen.tx[,chloro]
head(pg.tx)

### get a dataframe for each plot
plots <- uni(popgen.tx$pop)[1:3]
dfs <- c()
for (plot in plots){
    df <- popgen.tx[which(popgen.tx$pop == plot),]
    assign(sprintf("%spop",plot),df$pop)
    df <- df[,chloro]
    p <- plot
    assign(p,df)
}

### get genind objects
geninds <- c()
for (plot in plots){
    df      <- get(plot)
    pops    <- get(sprintf("%spop",plot))
    genind  <- sprintf("gen%s",plot)
    geninds <- c(geninds,genind)
    assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}
genuc
genbc
genun

### get gd objects
gdnames = c()
for (genind in geninds){
    gdname <- sprintf("gd%s",genind)
    gdnames <- c(gdnames,gdname)
    gd <- gd.smouse(get(genind))
    gd <- as.matrix(gd)
    assign(gdname,gd)
}
# check dimensions
for (gdn in gdnames){
    gd <- get(gdn)
    print(gdn)
    print(dim(gd))
}

### get euclidian distance
len(which(is.na(distmat))) == 0

### from full distmat, create smaller distmats based on treatment type and size class
simpmoran[,'plot']      <- substr(simpmoran$samp,1,3)
simpmoran[,'treatment'] <- substr(simpmoran$plot,1,2)
simpmoran[,'plot_rep']  <- substr(simpmoran$plot,3,3)

txs <- uni(simpmoran$treatment)[1:3]
distnames <- c()
for (tx in txs){
    print(tx)
    gdname <- sprintf("gdgen%s",tx)
    gd <- get(gdname)
    sampnames <- rownames(gd) # adults in treatment
    d <- distmat[which(rownames(distmat) %in% sampnames),which(colnames(distmat) %in% sampnames)]
    print(dim(d))
    distname  <- sprintf('dist%s',tx)
    distnames <- c(distnames,distname)
    assign(distname,d)
}


### fill in NAs for adultxadult and seedxseed
seedlings <- simpmoran[which(simpmoran[,'seedling'] == 'True'),'samp'] # should be 763 (noff)
adults    <- simpmoran[which(simpmoran[,'seedling'] == 'False'),'samp']   # should be 599 (nadult)
getclass = function(samp){
    if (samp %in% adults){
        return('adult')
    }
    if (samp %in% seedlings){
        return('seedling')
    }
    stop('couldnt find class',call.=FALSE)
}
for (dist in distnames){
    print(dist)
    d <- get(dist)
    for (i in 1:nrow(d)){
        sampi = rownames(d)[i]
        aorsi = getclass(sampi)
        for (j in 1:ncol(d)){
            sampj = colnames(d)[j]
            aorsj = getclass(sampj)
            if (i > j){
                if (aorsi == aorsj){
                    d[i,j] <- NA
                    d[j,i] <- NA
                }
            }
        }
    }
    diag(d) <- 0
    assign(dist,d)
}
distbc[1:50,1:50]
distun[1:50,1:50]
distuc[1:50,1:50]

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
    print(gd)
    gdist <- get(gd)
    tx <- substr(gd,6,7)
    d <- get(sprintf("dist%s",tx))
    bins = ceiling(.max(d)/10)
    binname = sprintf("bin%s",gd)
    assign(binname,bins)
    autobins <- c(autobins,binname)
    auto <- spautocor(gdist,d,bins=bins)
    autoname <- sprintf("auto%s",gd)
    autos <- c(autos,autoname)
    assign(autoname,auto)
}

### print out range in r
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (range(a$r))
}
for (auto in autos){
    print (auto)
    a <- get(auto)
    print (.range(a$r))
}


### save environment
DIR = '~/fragment/postmoran/by_tx/size_class/chloro/adultxseed'
if (!(file.exists(DIR))){
    dir.create(DIR)
    print('created DIR')
}
cd(DIR)
rds()
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp ~/fragment/postmoran/by_tx/size_class/chloro/adultxseed/*RDS godel:/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/final

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed'
dirs = ['bc',
        'uc',
        'un',
        'us']
for d in dirs:
    d = op.join(DIR,d)
    if not op.exists(d):
        mkdir(d)
dirs = []
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed'
ds = ['bc',
        'uc',
        'un',
        'us']
for d in ds:
    dirs.append(op.join(DIR,d))
    assert op.exists(op.join(DIR,d))
dirs

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# write scripts per tx to get 999 null values of spatial autocorrelation
for tx in txs:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % tx,
           'dist%s' % tx,
           'bingdgen%s' % tx,
           tx,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/rfiles/null_%s.R' % (tx,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil
# get .R files
rfiles = [] # redundant so I don't use wrong rfiles
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
assert len(rfiles) == 2997

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/shfiles')
cwd()

In [ ]:
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/shfiles')
!ls *sh | parallel qsub {}

In [ ]:
# check on progress
for tx in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/outfiles/' % tx
    assert op.exists(DIR)
    print tx,len(ls(DIR))

In [ ]:
# combine null runs into a couple objects
for tx in txs:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/final')                      ###############
getrds()
tx = '%s'

sauto <- get(sprintf('autogdgen%%s',tx))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%%s/outfiles',tx) ###############
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',tx))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%%s',tx))       ####################
filE = fname(getwd(),sprintf('%%s_chloro_null.RDS',tx))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_chloro_R.RDS',tx))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_chloro_Rm.RDS',tx))
saveRDS(Rm,filE)

''' % (tx)
    filE = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/%s/%s_combine_R_vector_chloro.R' % (tx,tx)
    rfiles.append(f)
rfiles

In [ ]:
shfiles = []
for f in rfiles:
    tx = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (tx,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed/combine'
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % tx)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
cd(op.dirname(shfiles[0]))
cwd()

In [ ]:
for s in shfiles:
    cd(op.dirname(s))
    !qsub $s

In [ ]:
# scp these files to computer
DIR = '/home/lindb/teakettle/postmoran/by_tx/size_class/chloro/adultxseed'
prefs = ['autogdgen','bingdgen']
files = []
for tx in txs:
    d = op.join(DIR,tx)
    assert op.exists(d)
    for p in prefs:
        filE = op.join(op.join(DIR,'final'),'%s%s.RDS' % (p,tx))
        assert op.exists(filE)
        files.append(filE)
    filE = op.join(d,'%s_chloro_null.RDS' % tx)
    assert op.exists(filE)
    files.append(filE)
files

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/chloro_by_tx_adultxseed')
getrds()
txs <- c('bc','uc','un')

Lows <- list()
His  <- list()
for (tx in txs){
    null <- get(sprintf('%s_chloro_null',tx))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- tx
    names(His)[len(His)]   <- tx
}
```
### then use code below to plot figs

In [ ]:
##### print out text to paste into R
for tx in txs:
    text = '''
########
tx = '%s'
his  <- His[[sprintf('%%s',tx)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',tx)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',tx))
sauto[which(is.na(sauto$r)),'r'] <- 0
cutoff <- which(sauto$bin > 325)
dev.new(width=3.25,height=3.25)
par(cex.lab=1,cex.axis=1)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="",
    ylab="",
    ylim=c(-0.2,0.2),
    xaxt="n",
    main="")
xtick <- seq(0,max(sauto$bin),200)
axis(side=1,at=xtick)
#polygon(c(rev(sauto$bin[-cutoff]),sauto$bin[-cutoff]),c(rev(his[-cutoff]),lows[-cutoff]),col='grey75',border=FALSE)
#lines(sauto$bin,sauto$r,lwd=2)
#lines(sauto$bin[-cutoff],lows[-cutoff],col="red",lty=2)
#lines(sauto$bin[-cutoff],his[-cutoff],col="red",lty=2)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
####
dev.new(width=10,height=10)
par(cex.lab=1.5,cex.axis=1.5)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.2,0.2),
    xlim=c(0,300),
    xaxt="n",
    xaxt='n') #    main=sprintf('%%s',tx),
xtick <- seq(0,max(sauto$bin),25)
axis(side=1,at=xtick,FALSE)
axis(side=1,at=xtick[xtick %%%% 50 ==0],labels=xtick[xtick %%%% 50 ==0])
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col=grey.colors(15)[5],border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
abline(v=xtick,col='grey',lwd=0.5)
########
''' % tx
    print text

In [ ]:
# scp ~/fragment/postmoran/by_tx_chloro_adultxseed_* godel:/home/lindb/teakettle/postmoran/figs

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_bc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_bc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_uc.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_uc_inset.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_un.pdf')
img

In [ ]:
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/by_tx_chloro_adultxseed_un_inset.pdf')
img